In [26]:
import pandas as pd
import numpy as np
import seaborn as sb

data = pd.read_pickle('Binning.pickle')

data


,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,JOININGYEAR_BIN,AGE_BIN,EXPERIENCEINCURRENTDOMAIN_BIN
1,Bachelors,2013,C,1,28,Female,No,3,1,"(2011.999, 2013.0]","(27.0, 28.0]","(2.0, 3.0]"
2,Bachelors,2014,B,3,38,Female,No,2,0,"(2013.0, 2014.0]","(34.0, 38.0]","(1.0, 2.0]"
3,Masters,2016,A,3,27,Male,No,5,1,"(2015.0, 2016.0]","(26.0, 27.0]","(4.0, 5.0]"
5,Bachelors,2016,A,3,22,Male,No,0,0,"(2015.0, 2016.0]","(21.999, 24.7]","(-0.001, 1.0]"
6,Bachelors,2015,B,3,38,Male,No,0,0,"(2014.0, 2015.0]","(34.0, 38.0]","(-0.001, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
4647,Bachelors,2016,C,3,30,Male,No,2,0,"(2015.0, 2016.0]","(28.0, 31.0]","(1.0, 2.0]"
4648,Bachelors,2013,A,3,26,Female,No,4,0,"(2011.999, 2013.0]","(24.7, 26.0]","(3.0, 4.0]"
4649,Masters,2013,C,2,37,Male,No,2,1,"(2011.999, 2013.0]","(34.0, 38.0]","(1.0, 2.0]"
4650,Masters,2018,B,3,27,Male,No,5,1,"(2017.0, 2018.0]","(26.0, 27.0]","(4.0, 5.0]"


In [27]:
def num_event(event, dataframe ,field, val):
    if event == 'good':
        event = 1
    else:
        event = 0
    cond_main = dataframe[field] == val
    cond_event = dataframe['LeaveOrNot'] == event
    n = len(dataframe[cond_main & cond_event].index)
    return n


def dist_event(event, dataframe ,field, val):
    n = num_event(event, dataframe, field, val)
    sumd = 0
    all_elements = set(dataframe[field].to_numpy())
    for a in all_elements:
        s = num_event(event, dataframe, field, a)
        sumd = sumd + s
    return n/sumd

def WOE(dataframe, field, val):
    dg = dist_event('good', dataframe, field, val)
    db = dist_event('bad', dataframe, field, val)
    w = np.log(dg / db)
    return w
dt_train = pd.DataFrame()
dt_train['LeaveOrNot'] = data['LeaveOrNot']
ATTRS = ['Education', 'PaymentTier', 'AGE_BIN', 'Gender', 'EverBenched','JOININGYEAR_BIN','EXPERIENCEINCURRENTDOMAIN_BIN']


NEW_WOE_ATTRS=[]
for attr in ATTRS:
    attr_element = set(data[attr].to_numpy())
    attr_woe = dict()
    for i in attr_element:
        w = WOE(data, attr, i)
        attr_woe[i] = w
        NEW_WOE_ATTRS.append( attr_woe[i])
    print(f'WOE {attr} -> {attr_woe}')
    

    lst = []
    for s in data[attr]:
        woe = attr_woe[s]
        lst.append(woe)

    new_field = attr + '_WOE'
    dt_train[new_field] = lst


dt_train



WOE Education -> {'PHD': -0.4709699310516477, 'Masters': 0.6615439092921434, 'Bachelors': -0.15886505087872912}
WOE PaymentTier -> {1: 0.04773347096871903, 2: 1.004973626480855, 3: -0.30473556270485164}
WOE AGE_BIN -> {Interval(34.0, 38.0, closed='right'): -0.1291464255764391, Interval(31.0, 34.0, closed='right'): -0.11257233380146908, Interval(28.0, 31.0, closed='right'): -0.13338596557774418, Interval(38.0, 41.0, closed='right'): -0.13295576552808946, Interval(26.0, 27.0, closed='right'): 0.13805988865422922, Interval(24.7, 26.0, closed='right'): 0.1486152353293741, Interval(21.999, 24.7, closed='right'): 0.21242617102624228, Interval(27.0, 28.0, closed='right'): -0.1931889295825783}
WOE Gender -> {'Male': -0.3859132160750363, 'Female': 0.5076309567927609}
WOE EverBenched -> {'No': -0.06996982236309017, 'Yes': 0.5458079572574848}
WOE JOININGYEAR_BIN -> {Interval(2011.999, 2013.0, closed='right'): -0.21810019947223716, Interval(2013.0, 2014.0, closed='right'): -0.5272194763702639, Int

,LeaveOrNot,Education_WOE,PaymentTier_WOE,AGE_BIN_WOE,Gender_WOE,EverBenched_WOE,JOININGYEAR_BIN_WOE,EXPERIENCEINCURRENTDOMAIN_BIN_WOE
1,1,-0.158865,0.047733,-0.193189,0.507631,-0.069970,-0.218100,0.212885
2,0,-0.158865,-0.304736,-0.129146,0.507631,-0.069970,-0.527219,0.095246
3,1,0.661544,-0.304736,0.138060,-0.385913,-0.069970,-0.711505,-0.116709
5,0,-0.158865,-0.304736,0.212426,-0.385913,-0.069970,-0.711505,-0.055357
6,0,-0.158865,-0.304736,-0.129146,-0.385913,-0.069970,0.231233,-0.055357
...,...,...,...,...,...,...,...,...
4647,0,-0.158865,-0.304736,-0.133386,-0.385913,-0.069970,-0.711505,0.095246
4648,0,-0.158865,-0.304736,0.148615,0.507631,-0.069970,-0.218100,-0.128677
4649,1,0.661544,1.004974,-0.129146,-0.385913,-0.069970,-0.218100,0.095246
4650,1,0.661544,-0.304736,0.138060,-0.385913,-0.069970,4.612788,-0.116709


In [28]:
Y =  dt_train['LeaveOrNot'].to_numpy()
x_1 = dt_train['Education_WOE'].to_numpy()
x_2 = dt_train['PaymentTier_WOE'].to_numpy()
x_3 = dt_train['AGE_BIN_WOE'].to_numpy()
x_4 = dt_train['Gender_WOE'].to_numpy()
x_5 = dt_train['EverBenched_WOE'].to_numpy()
x_6 = dt_train['JOININGYEAR_BIN_WOE'].to_numpy()
x_7 = dt_train['EXPERIENCEINCURRENTDOMAIN_BIN_WOE'].to_numpy()



array_ones = np.ones(x_1.shape)

X = np.array([array_ones, x_1,x_2,x_3,x_4,x_5,x_6,x_7])
X = np.transpose(X)
X


array([[ 1.        , -0.15886505,  0.04773347, ..., -0.06996982,
        -0.2181002 ,  0.21288519],
       [ 1.        , -0.15886505, -0.30473556, ..., -0.06996982,
        -0.52721948,  0.09524601],
       [ 1.        ,  0.66154391, -0.30473556, ..., -0.06996982,
        -0.71150523, -0.11670928],
       ...,
       [ 1.        ,  0.66154391,  1.00497363, ..., -0.06996982,
        -0.2181002 ,  0.09524601],
       [ 1.        ,  0.66154391, -0.30473556, ..., -0.06996982,
         4.61278763, -0.11670928],
       [ 1.        , -0.15886505, -0.30473556, ...,  0.54580796,
        -0.2181002 ,  0.09524601]])

In [29]:
np.matmul(np.transpose(X), X)


array([[ 3.25800000e+03, -4.60692612e+01, -9.40900754e+01,
        -1.22998623e+01, -9.39107449e+01, -1.49025695e+01,
         3.78610528e+02, -9.84666665e+00],
       [-4.60692612e+01,  3.61307507e+02,  1.49615389e+02,
         3.67849163e+00,  7.69032584e+00,  8.18382190e-01,
        -2.51915855e+00,  8.57380126e+00],
       [-9.40900754e+01,  1.49615389e+02,  8.75198187e+02,
         3.31709080e+00,  1.98046634e+02,  2.60727870e+00,
        -2.24630977e+02,  6.01094554e+00],
       [-1.22998623e+01,  3.67849163e+00,  3.31709080e+00,
         7.76398190e+01, -4.03195279e-01, -4.98666574e-01,
         1.26101908e+01, -4.39509775e+00],
       [-9.39107449e+01,  7.69032584e+00,  1.98046634e+02,
        -4.03195279e-01,  6.26816467e+02, -1.37108158e+00,
        -2.49052800e+01,  2.27768910e+00],
       [-1.49025695e+01,  8.18382190e-01,  2.60727870e+00,
        -4.98666574e-01, -1.37108158e+00,  1.17332089e+02,
         1.03753803e+02,  3.89943948e-01],
       [ 3.78610528e+02, -2.519158

In [30]:
np.linalg.inv(np.matmul(np.transpose(X), X))


array([[ 3.11870623e-04,  3.15079979e-05,  1.29607822e-05,
         5.40032622e-05,  4.14334834e-05,  5.71714312e-05,
        -1.98140852e-05,  5.02052840e-05],
       [ 3.15079979e-05,  3.00391182e-03, -5.43818347e-04,
        -1.31116871e-04,  1.40215666e-04,  1.45212576e-05,
        -1.93087198e-05, -3.70578041e-04],
       [ 1.29607822e-05, -5.43818347e-04,  1.34368986e-03,
        -4.28530383e-05, -4.14027297e-04, -7.17185362e-05,
         4.80016716e-05, -5.04653627e-05],
       [ 5.40032622e-05, -1.31116871e-04, -4.28530383e-05,
         1.29574123e-02,  2.68893194e-05,  9.19408641e-05,
        -3.50849920e-05,  9.60918757e-04],
       [ 4.14334834e-05,  1.40215666e-04, -4.14027297e-04,
         2.68893194e-05,  1.73044424e-03,  4.40008303e-05,
        -1.13908999e-05, -3.24949198e-05],
       [ 5.71714312e-05,  1.45212576e-05, -7.17185362e-05,
         9.19408641e-05,  4.40008303e-05,  8.66784162e-03,
        -1.53063617e-04, -4.47645943e-06],
       [-1.98140852e-05, -1.930871

In [31]:
np.matmul(np.transpose(X), Y)


array([1109.        ,   69.0943947 ,  173.98487989,   13.24387004,
        109.04229127,   22.77524508, 1010.64821561,   10.30638937])

In [32]:
B = np.matmul(np.linalg.inv(np.matmul(np.transpose(X), X)), np.matmul(np.transpose(X), Y))
B

array([0.3373242 , 0.13842969, 0.21122568, 0.19445175, 0.16180612,
       0.10061673, 0.150259  , 0.1577975 ])

In [33]:
def predict(x):
    y = B[0] + B[1]*x+ B[2]*x+ B[3]*x+ B[4]*x+ B[5]*x \
    + B[6]*x+ B[7]*x
    return y

eq = f'y = {B[0]:.3f}'
for i in range(1, len(B)):
    eq = eq + f'+ ({B[i]:.3f})x_{i}'
print(eq)

y = 0.337+ (0.138)x_1+ (0.211)x_2+ (0.194)x_3+ (0.162)x_4+ (0.101)x_5+ (0.150)x_6+ (0.158)x_7


In [34]:
data_test = pd.read_pickle('Data_test.pickle')

attr = pd.read_pickle('attr.pickle')
attr


,ATTR,VAL,N_GOOD,N_BAD,DIST_GOOD,DIST_BAD,WOE
0,AGE_BIN,"(21.999, 24.7]",127,199,0.114518,0.092601,0.212426
1,AGE_BIN,"(24.7, 26.0]",285,476,0.256988,0.221498,0.148615
2,AGE_BIN,"(26.0, 27.0]",157,265,0.141569,0.123313,0.138060
3,AGE_BIN,"(27.0, 28.0]",134,315,0.120830,0.146580,-0.193189
4,AGE_BIN,"(28.0, 31.0]",126,279,0.113616,0.129828,-0.133386
5,AGE_BIN,"(31.0, 34.0]",83,180,0.074842,0.083760,-0.112572
6,AGE_BIN,"(34.0, 38.0]",122,269,0.110009,0.125174,-0.129146
7,AGE_BIN,"(38.0, 41.0]",75,166,0.067628,0.077245,-0.132956
8,EXPERIENCEINCURRENTDOMAIN_BIN,"(-0.001, 1.0]",208,426,0.187556,0.198232,-0.055357
9,EXPERIENCEINCURRENTDOMAIN_BIN,"(1.0, 2.0]",277,488,0.249775,0.227082,0.095246


In [35]:
# y_predict
data_test['y_predict'] = B[0] + B[1]*dt_train['Education_WOE'] + B[2]*dt_train['PaymentTier_WOE'] \
        + B[3]*dt_train['AGE_BIN_WOE'] + B[4]*dt_train['Gender_WOE'] + B[5]*dt_train['EverBenched_WOE'] \
        + B[6]*dt_train['JOININGYEAR_BIN_WOE'] + B[7]*dt_train['EXPERIENCEINCURRENTDOMAIN_BIN_WOE']

data_test['P'] = 1/(1+np.exp(-data_test['y_predict']))

data_test['predict'] = data_test['P'].map(lambda x: 1 if x >= 0.5 else 0)
data_test

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,y_predict,P,predict
0,Bachelors,2017,A,3,34,Male,No,0,0,NaN,NaN,0
4,Masters,2017,C,3,24,Male,Yes,2,1,NaN,NaN,0
11,Bachelors,2016,C,3,34,Male,No,3,0,NaN,NaN,0
14,Bachelors,2012,A,3,37,Male,No,4,0,NaN,NaN,0
15,Bachelors,2017,A,1,29,Male,No,3,0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4634,Bachelors,2016,B,1,38,Female,No,2,1,NaN,NaN,0
4637,Bachelors,2018,A,3,32,Male,Yes,2,1,NaN,NaN,0
4638,Bachelors,2013,A,3,26,Male,No,4,0,NaN,NaN,0
4644,Bachelors,2015,C,3,32,Female,Yes,1,1,NaN,NaN,0


In [36]:
n_correct = (data_test['LeaveOrNot'] == data_test['predict']).sum()
print(n_correct)

accuracy = n_correct/1395
print('accuracy is')
print(accuracy)


904
accuracy is
0.6480286738351254
